<a href="https://colab.research.google.com/github/PKpacheco/project4_intro_ai/blob/main/(spam)Project_4_Intro_AI_Paola_Pacheco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Project 4 Spam Classifier (Part 2)
Intro to AI
\
Instructor: Jeffrey Lanyon
\
Student: Paola Katherine Pacheco


# Import

In [1]:
# import libraries

import pandas as pd
import numpy as np

from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer

from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from keras.utils import to_categorical


# 1 Data Loading and Preparation
###Load and concatenate data from multiple CSV files using Pandas.

Load comments from five CSV files named Youtube01-Psy.csv, Youtube02-
KatyPerry.csv, Youtube03-LMFAO.csv, Youtube04-Eminem.csv, and Youtube05-
Shakira.csv.


In [2]:
# get files url
url_psy = 'https://raw.githubusercontent.com/PKpacheco/intro_ai_project_3/main/Youtube01-Psy.csv'
url_kateperry = 'https://raw.githubusercontent.com/PKpacheco/intro_ai_project_3/main/Youtube02-KatyPerry.csv'
url_lmfao = 'https://raw.githubusercontent.com/PKpacheco/intro_ai_project_3/main/Youtube03-LMFAO.csv'
url_eminem = 'https://raw.githubusercontent.com/PKpacheco/intro_ai_project_3/main/Youtube04-Eminem.csv'
url_shakira = 'https://raw.githubusercontent.com/PKpacheco/intro_ai_project_3/main/Youtube05-Shakira.csv'

In [3]:
# read files
df_kate = pd.read_csv(url_kateperry)
df_lmfao = pd.read_csv(url_lmfao)
df_eminem = pd.read_csv(url_eminem)
df_sharika = pd.read_csv(url_shakira)
df_psy = pd.read_csv (url_psy)

In [4]:
# all df
all_dataframes = [
    df_psy,
    df_kate,
    df_lmfao,
    df_eminem,
    df_sharika
    ]

# concatenate into 1 df
concat_df = pd.concat(all_dataframes, ignore_index=True)

In [5]:
# shuffle
shuffled_df = concat_df.sample(frac=1, random_state=42).reset_index(drop=True)
print(shuffled_df.head())

                                    COMMENT_ID  \
0          z120djlhizeksdulo23mj5z52vjmxlhrk04   
1            z133ibkihkmaj3bfq22rilaxmp2yt54nb   
2        z12gxdortqzwhhqas04cfjrwituzghb5tvk0k   
3  _2viQ_Qnc6_ZYkMn1fS805Z6oy8ImeO6pSjMLAlwYfM   
4        z120s1agtmmetler404cifqbxzvdx15idtw0k   

                            AUTHOR                        DATE  \
0             Murlock Nightcrawler  2015-05-24T07:04:29.844000   
1  Debora Favacho (Debora Sparkle)  2015-05-21T14:08:41.338000   
2             Muhammad Asim Mansha                         NaN   
3                      mile panika  2013-11-03T14:39:42.248000   
4                   Sheila Cenabre         2014-08-19T12:33:11   

                                             CONTENT  CLASS  
0                                Charlie from LOST?﻿      0  
1                        BEST SONG EVER X3333333333﻿      0  
2                     Aslamu Lykum... From Pakistan﻿      1  
3  I absolutely adore watching football plus I’ve...  

In [6]:
shuffled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1956 entries, 0 to 1955
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   COMMENT_ID  1956 non-null   object
 1   AUTHOR      1956 non-null   object
 2   DATE        1711 non-null   object
 3   CONTENT     1956 non-null   object
 4   CLASS       1956 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 76.5+ KB


In [7]:
shuffled_df

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,z120djlhizeksdulo23mj5z52vjmxlhrk04,Murlock Nightcrawler,2015-05-24T07:04:29.844000,Charlie from LOST?﻿,0
1,z133ibkihkmaj3bfq22rilaxmp2yt54nb,Debora Favacho (Debora Sparkle),2015-05-21T14:08:41.338000,BEST SONG EVER X3333333333﻿,0
2,z12gxdortqzwhhqas04cfjrwituzghb5tvk0k,Muhammad Asim Mansha,NaN,Aslamu Lykum... From Pakistan﻿,1
3,_2viQ_Qnc6_ZYkMn1fS805Z6oy8ImeO6pSjMLAlwYfM,mile panika,2013-11-03T14:39:42.248000,I absolutely adore watching football plus I’ve...,1
4,z120s1agtmmetler404cifqbxzvdx15idtw0k,Sheila Cenabre,2014-08-19T12:33:11,I really love this video.. http://www.bubblews...,1
...,...,...,...,...,...
1951,z121szzyozr4vpqqc04cdn5g4zjhutdosdw,ItsJoey Dash,2014-07-22T10:04:05.755000,EVERYONE PLEASE SUBSCRIBE TO MY CHANNEL OR CAN...,1
1952,LneaDw26bFvtmJorLbUQjuqmpL7TdjOKPGJT_DGUUJ4,louis canellony,NaN,watch youtube video &quot;EMINEM -YTMA artist ...,1
1953,z13cfr5o3mnyez4ju04ce3whmvflg1fifl00k,vieshva .d.exodous,2015-05-18T08:38:34.236000,Awesome﻿,0
1954,z12rhdkrvvy3hd42c23whdeavkjdxnimg,viviane trinh,2015-05-21T22:35:35.753000,i like the lyrics but not to music video﻿,0


#2 Setting Up cross-validation
###Implement Stratified K-Fold Cross-Validation.

* Use the StratifiedKFold class from sklearn.model_selection to create cross-
validation splits. Set the number of splits to 5.
*  Ensure that the splits are stratified based on the CLASS column in your DataFrame.
* Output the pair train,test indices from your splits in the previous task. train
contains the indices of the dataset that should be used for training in this particular split. test
contains the indices of the dataset that should be used for testing in this particular split. The loop
should print “split to indicate the start of a new split formation, then prints test indices. Sample
output is given by

In [8]:
stratify_column = 'CLASS'

X = shuffled_df.drop(columns=[stratify_column])
y = shuffled_df[stratify_column]

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [9]:
X.shape

(1956, 4)

In [10]:
y.shape

(1956,)

In [11]:
train_indices_list = []
test_indices_list = []

for split, (train_idx, test_idx) in enumerate(stratified_kfold.split(X, y), start=1):
    print(f"Split {split}")
    print("Train indices:", train_idx)
    print("Test indices:", test_idx)
    print("\n")
    train_indices_list.append(train_idx)
    test_indices_list.append(test_idx)


Split 1
Train indices: [   0    1    2 ... 1952 1953 1954]
Test indices: [  10   20   25   28   31   40   41   47   48   52   60   64   71   73
   80   81   85   86   87   89   90   95   99  101  104  106  113  114
  116  121  126  132  137  146  149  152  156  174  178  183  195  214
  231  234  246  254  263  265  268  270  277  284  293  303  318  339
  342  350  357  360  371  373  377  379  392  398  402  405  410  416
  418  419  426  427  431  432  435  438  444  446  447  449  458  459
  467  472  483  484  486  493  495  497  498  509  513  517  518  521
  529  538  545  549  565  574  575  576  594  610  611  612  613  619
  623  625  626  631  636  643  645  652  653  655  661  669  671  674
  675  679  681  682  685  686  687  693  697  700  701  702  704  708
  710  712  719  721  729  735  740  741  745  748  753  755  756  757
  770  783  785  791  794  805  810  811  812  814  817  842  852  858
  860  863  867  873  874  884  885  890  891  895  916  918  923  924
  92

In [12]:
train_idx.shape


(1565,)

In [13]:
test_idx.shape

(391,)

#3 Tokenization and Text-to-Matrix Conversion
###Convert text data into a numerical format suitable for a neural network.
* Create a function prepare_data that takes indices for training and testing data.
* Use the Tokenizer from keras.preprocessing.text to tokenize the text comments.
* Limit the number of words to 2000.
* Convert the text comments to a TF-IDF matrix format

In [14]:
def prepare_data(train_indices, test_indices, texts):

    train_data = texts.iloc[train_indices]
    test_data = texts.iloc[test_indices]

    #init tokenizer with num_words=2000
    tokenizer = Tokenizer(num_words=2000)

    tokenizer.fit_on_texts(train_data['CONTENT'])

    tfidf_vectorizer = TfidfVectorizer(max_features=2000)
    X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['CONTENT'])
    X_test_tfidf = tfidf_vectorizer.transform(test_data['CONTENT'])

    return X_train_tfidf, X_test_tfidf


    # # X_train = tokenizer.texts_to_matrix(train_data['CONTENT'], mode='tfidf')
    # # X_test = tokenizer.texts_to_matrix(test_data['CONTENT'], mode='tfidf')

    # # return X_train, X_test

    # return X_matrix


In [15]:
# define indices and call function

train_indices = train_indices_list[1]
test_indices = test_indices_list[1]

X_train_tfidf, X_test_tfidf = prepare_data(train_indices, test_indices, shuffled_df)


In [16]:
X_train_tfidf.shape


(1565, 2000)

In [17]:
X_test_tfidf.shape

(391, 2000)

#4 Data Preprocessing
### Task: Normalize the TF-IDF matrix.
* Divide the matrix by its maximum absolute value.
* Subtract the mean from the matrix

In [18]:
# Normalize using max abs value
max_abs_scaler = MaxAbsScaler()

X_train_normalized = max_abs_scaler.fit_transform(X_train_tfidf)
X_test_normalized = max_abs_scaler.transform(X_test_tfidf)

# matrix - mean
mean_value = X_train_normalized.mean(axis=0)
X_train_normalized -= mean_value
X_test_normalized -= mean_value

print("Shape of X_train_normalized:", X_train_normalized.shape)
print("Shape of X_test_normalized:", X_test_normalized.shape)


Shape of X_train_normalized: (1565, 2000)
Shape of X_test_normalized: (391, 2000)


# 5 Building the Neural Network Model
### Task: Define and compile a neural network model.
* Use the Sequential model from Keras.
* Add a Dense layer with 512 units and 'relu' activation, followed by a Dropout layer with
0.5 dropout rate.
* Add another Dense layer for classification and use 'softmax' activation.
* Compile the model with 'categorical_crossentropy' loss and 'adamax' optimizer

In [19]:
input_shape = X_train_normalized.shape[1]
input_shape

2000

In [20]:
# function to create the model
def create_model(input_shape, num_classes):
    # sequential model
    model = Sequential()

    # Dense layer with 512 units and 'relu' activation
    model.add(Dense(512, activation='relu', input_shape=(input_shape,)))

    # dropout layer with 0.5 dropout rate
    model.add(Dropout(0.5))

    # another dense layer 'softmax' activation
    model.add(Dense(num_classes, activation='softmax'))

    # complile the model
    model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])

    # model summary
    model.summary()

    return model

# 6 Model Training and Evaluation
### Task: Train and evaluate the model.
* Write a function train_and_test that takes training and testing indices.
* Train the model on the training data and evaluate it on the test data.
* Use the accuracy metric to evaluate the model's performance

In [21]:
accuracy_scores = []

def train_and_test(train_indices, test_indices, X_train, y_train, X_test, y_test, epochs=10, batch_size=32):
    input_shape = X_train.shape[1]
    num_classes = 2

    # Create the model
    model = create_model(input_shape, num_classes)

    # Convert labels to categorical format
    y_train_categorical = to_categorical(y_train, num_classes=num_classes)
    y_test_categorical = to_categorical(y_test, num_classes=num_classes)

    # training model
    history = model.fit(X_train, y_train_categorical, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.1)

    # evaluate with test data
    results_model = model.evaluate(X_test, y_test_categorical, verbose=0)

    # loss and accuracy for each epoch
    for epoch, acc in enumerate(history.history['accuracy']):
        print(f'Epoch {epoch + 1} - Loss: {history.history["loss"][epoch]}, Accuracy: {acc}')
    # Print accuracy
    print(f'\nTest Accuracy: {results_model[1]*100:.2f}%')
    return results_model[1]

In [22]:
# Based on StratifiedKFold get split number 1
split_number = 1
train_indices = train_indices_list[split_number]
test_indices = test_indices_list[split_number]


In [23]:
accuracy = train_and_test(train_indices, test_indices, X_train_normalized, y.iloc[train_indices], X_test_normalized, y.iloc[test_indices])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               1024512   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 1025538 (3.91 MB)
Trainable params: 1025538 (3.91 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
44/44 [==============================] - 3s 38ms/step - loss: 0.6084 - accuracy: 0.7713 - val_loss: 0.5283 - val_accuracy: 0.9045
Epoch 2/10
44/44 [==============================] - 1s 23ms/step - loss: 0.4399 - accuracy: 0.9261 - val_loss: 0.4010 - val_accuracy: 0.9108


In [24]:
accuracy_scores.append(accuracy)


In [25]:
accuracy_scores

[0.9437339901924133]

#7  Cross-Validation Scores
### Task: Calculate and print the mean and standard deviation of the cross-validation scores.
* Run the train_and_test function for each split in the cross-validation.
* Collect the accuracy scores from each run and calculate their mean and standard
deviation. Print the loss and accuracy output for each of the 10 epochs. Sample output for
a fold should look as follows
* Print the mean and standard deviation of all of these scores

In [26]:
train_indices_list

[array([   0,    1,    2, ..., 1952, 1953, 1954]),
 array([   1,    2,    3, ..., 1953, 1954, 1955]),
 array([   0,    4,    5, ..., 1952, 1953, 1955]),
 array([   0,    1,    2, ..., 1953, 1954, 1955]),
 array([   0,    1,    2, ..., 1952, 1954, 1955])]

In [27]:
for split_number in range(1, 5):
    train_indices = train_indices_list[split_number]
    test_indices = test_indices_list[split_number]

    # each split, train and store the accuracy
    accuracy = train_and_test(train_indices, test_indices, X_train_normalized, y.iloc[train_indices], X_test_normalized, y.iloc[test_indices])

    # Append accuracy score for this split
    accuracy_scores.append(accuracy)

    print(f"Accuracy for Split {split_number}: {accuracy}")


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               1024512   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 1025538 (3.91 MB)
Trainable params: 1025538 (3.91 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
44/44 [==============================] - 3s 40ms/step - loss: 0.5968 - accuracy: 0.8182 - val_loss: 0.5218 - val_accuracy: 0.8471
Epoch 2/10
44/44 [==============================] - 1s 20ms/step - loss: 0.4249 - accuracy: 0.9297 - val_loss: 0.3972 - val_accuracy: 0.891

In [28]:
accuracy_scores

[0.9437339901924133,
 0.9437339901924133,
 0.5447570085525513,
 0.529411792755127,
 0.5140665173530579]

In [29]:
#  mean and std dev for accuracy
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

print(f"\nMean Accuracy: {mean_accuracy*100:.2f}%")
print(f"Standard Deviation of Accuracy: {std_accuracy*100:.2f}%")


Mean Accuracy: 69.51%
Standard Deviation of Accuracy: 20.32%


The mean accuracy is 69%, while the standard deviation is 20%, this is how much the accuracy values deviate from the mean.
This standard deviation is high, there is a variation in performance in the cross evaluation depending on the split.
We see that the accuracy varies from 50.69% to 94.63%, the model is sensitive and shows different results with the different types of data from each split